In [1]:
import time
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import os
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import zipfile
import math
from torch import nn
import torch.optim as optim
from torchvision import datasets, transforms
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import save_image
import os


if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip "/content/drive/MyDrive/Eye_Glasses/Input.zip" -d "/content/images1"
!unzip "/content/drive/MyDrive/Eye_Glasses/Output.zip" -d "/content/images1"

Archive:  /content/drive/MyDrive/Eye_Glasses/Input.zip
 extracting: /content/images1/B4120C1_img_01.jpeg  
 extracting: /content/images1/B4120C1_img_02.jpeg  
 extracting: /content/images1/B4120C1_img_03.jpeg  
 extracting: /content/images1/B4120C1_img_04.jpeg  
 extracting: /content/images1/B4120C1_img_05.jpeg  
 extracting: /content/images1/B4120C1_img_06.jpeg  
 extracting: /content/images1/B4120C1_img_07.jpeg  
 extracting: /content/images1/B4120C1_img_08.jpeg  
Archive:  /content/drive/MyDrive/Eye_Glasses/Output.zip
 extracting: /content/masks1/B4120C1_img_01.jpeg  
 extracting: /content/masks1/B4120C1_img_02.jpeg  
 extracting: /content/masks1/B4120C1_img_03.jpeg  
 extracting: /content/masks1/B4120C1_img_04.jpeg  
 extracting: /content/masks1/B4120C1_img_05.jpeg  
 extracting: /content/masks1/B4120C1_img_06.jpeg  
 extracting: /content/masks1/B4120C1_img_07.jpeg  
 extracting: /content/masks1/B4120C1_img_08.jpeg  


In [4]:
# transformer=transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
# ])
transformer = transforms.Compose([
    transforms.Grayscale(), # Convert to single channel
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

batch_size=32

In [23]:
class GlassDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        image = read_image(image_path)
        mask = read_image(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        # Convert images and masks to float tensors
        image = ToTensor()(image)
        mask = ToTensor()(mask)

        return image, mask

# Define transformations
transforms = Compose([
    Resize((224, 224)),
    Grayscale(),
    ToTensor()
])

# Create the dataset
glass_dataset = GlassDataset('/content/images1', '/content/masks1', transform=transforms)


In [ ]:
dataset_path = '/content/input2'

def load_dataset():
    train_dataset_manual = torchvision.datasets.ImageFolder(dataset_path, transform=transformer)
    train_loader_manual = torch.utils.data.DataLoader(train_dataset_manual)
    return train_loader_manual


full_dataset = load_dataset()


train_size = int(0.6 * len(full_dataset))
test_size = int (0.2 * len(full_dataset))
valid_size = len(full_dataset) - train_size - test_size


train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(full_dataset.dataset, [train_size, test_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=0, shuffle=True)

print('Full Dataset - ' + str(len(full_dataset)) + ' images.')
print('Train Set- ' + str(train_size) + ' images in ' + str(len(train_loader)) +' batches')
print('Testing Set - ' + str(test_size) + ' images in ' + str(len(test_loader)) + ' batches' )
print('Validation Set - ' + str(valid_size) + ' images in ' + str(len(valid_loader)) + ' batches')

train_count = train_size
test_count = test_size
valid_count = valid_size

Full Dataset - 8 images.
Train Set- 4 images in 1 batches
Testing Set - 1 images in 1 batches
Validation Set - 3 images in 1 batches


In [ ]:
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([4, 1, 224, 224])
Image label dimensions: torch.Size([4])


In [11]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Downscaling
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Upscaling
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2)
        )

        # Final layer
        self.final_layer = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoding
        x1 = self.encoder(x)

        # Decoding
        x2 = self.decoder(x1)

        # Final output
        out = self.final_layer(x2)
        return out

# Example usage:
# Create a U-Net model with 1 input channel and 2 output classes
model2 = UNet(in_channels=1, out_channels=2)

In [12]:
from torchsummary import summary
summary(model2, (1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             640
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6         [-1, 64, 112, 112]          36,928
              ReLU-7         [-1, 64, 112, 112]               0
            Conv2d-8         [-1, 64, 112, 112]          36,928
              ReLU-9         [-1, 64, 112, 112]               0
  ConvTranspose2d-10          [-1, 2, 224, 224]             514
           Conv2d-11          [-1, 2, 224, 224]               6
Total params: 111,944
Trainable params: 111,944
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

In [ ]:
 epoch_list = []
 train_accuracy_list = []
 val_loss_list = []
 val_accuracy_list = []
 train_loss_list = []

In [ ]:
device = torch.cuda.is_available() if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
 for epoch in range(10):
    train_loss = 0.0
    train_correct = 0.0
    train_total = 0.0
    val_loss = 0.0
    val_correct = 0.0
    val_total = 0.0

    # Train the model
    model2.train()
    for i, data in enumerate(train_loader, 0):
        # Get the inputs and labels
        inputs, labels = data
        inputs, labels=inputs.to(device),labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Compute training loss
        train_loss += loss.item()

    # Evaluate the model on the validation set
    model2.eval()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels=images.to(device),labels.to(device)
            outputs = model2(images)
            loss = criterion(outputs, labels)

            # Compute validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            # Compute validation
            val_loss += loss.item()

    # Compute average training loss and accuracy
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    epoch_list.append(epoch)
    train_accuracy_list.append(train_accuracy)
    train_loss_list.append(train_loss)

    # Compute average validation loss and accuracy
    val_loss /= len(test_loader)
    val_accuracy = val_correct / val_total
    val_loss_list.append(val_loss)
    val_accuracy_list.append(val_accuracy)

    # Print the loss and accuracy values for each epoch
    print('Epoch [%d], Train Loss: %.4f, Train Acc: %.4f, Val Loss: %.4f, Val Acc: %.4f'
                    % (epoch + 1, train_loss, train_accuracy, val_loss, val_accuracy))

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

In [ ]:
print(labels.shape)  # Check the shape of the labels tensor

# If labels are one-hot encoded with shape [batch_size, num_classes, height, width]
if len(labels.shape) == 4:
    labels = torch.argmax(labels, dim=1)  # Convert to class indices with shape [batch_size, height, width]


torch.Size([4])


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # If your labels are one-hot encoded, convert them to class indices
        labels = torch.argmax(labels, dim=2)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model2(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()


        running_loss += loss.item()

    # Print statistics
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

    # Validation loop (optional)
    model2.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
        print(f'Validation Loss: {val_loss/len(valid_loader)}')


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 2)

In [16]:
import os
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor

# Define the custom Dataset class
class GlassDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        image = read_image(image_path)
        mask = read_image(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        # Convert images and masks to float tensors
        image = ToTensor()(image)
        mask = ToTensor()(mask)

        return image, mask

# Define transformations
transforms = Compose([
    Resize((224, 224)),
    Grayscale(),
    ToTensor()
])

# Create the dataset
glass_dataset = GlassDataset('/content/images1', '/content/masks1', transform=transforms)

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        # Downscaling
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Upscaling
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2)
        )

        # Final layer
        self.final_layer = nn.Conv2d(out_channels, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoding
        x1 = self.encoder(x)

        # Decoding
        x2 = self.decoder(x1)

        # Final output
        out = self.final_layer(x2)
        return out

# Example usage:
# Create a U-Net model with 1 input channel and 2 output classes
model2 = UNet(in_channels=1, out_channels=2)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model2.parameters(), lr=0.001)

# Create a DataLoader for your dataset
data_loader = DataLoader(glass_dataset, batch_size=4, shuffle=True)

# Define the number of epochs for training
num_epochs = 25

# Training loop
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0

    for images, masks in data_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model2(images)

        # Calculate loss
        loss = criterion(outputs, masks.squeeze(dim=1).long())

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print statistics
    epoch_loss = running_loss / len(data_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

print('Finished Training')


TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [ ]:
/